In [ ]:
import os
os.environ['APOGEE_CACHE_PATH'] = "/mnt/ceph/users/apricewhelan/apogee-test/"
os.environ['JOAQUIN_CACHE_PATH'] = "/mnt/ceph/users/apricewhelan/projects/joaquin/cache"
import warnings
warnings.filterwarnings('ignore', category=Warning) 
import pickle

import sys
import pathlib
_path = str(pathlib.Path('../').resolve())
if _path not in sys.path:
    sys.path.append(_path)

import corner
from astropy.io import fits
import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from tqdm.auto import tqdm
from sklearn.decomposition import PCA
from scipy.spatial import cKDTree

from joaquin.data import JoaquinData
from joaquin.config import root_cache_path, zone1_K, zone2_K
from joaquin.plot import simple_corner
from joaquin.neighborhoods import get_neighborhood_X

See: `PCA-neighbord-training.ipynb` and `PCA-neighbord-apply.ipynb`

In [ ]:
parent_projected_X = np.load(
    root_cache_path / 'parent_projected_X.npy')

zones_X = np.load(
    root_cache_path / 'neighborhoods_projected_X.npy')

spec_good_mask = np.load(root_cache_path / 'spec_good_mask.npy')

In [ ]:
parent_data = JoaquinData(
    cache_file='parent-sample')
parent_stars = parent_data.stars[parent_data.stars_mask]

In [ ]:
(X, *_), *_ = parent_data.get_Xy('spec', spec_mask_thresh=None)

In [ ]:
tree = cKDTree(parent_projected_X)

zone2_idx = []
for i in range(zones_X.shape[0]):
    dist, idx = tree.query(zones_X[i], k=zone2_K)
    zone2_idx.append(idx[dist.argsort()])
    
zone1_idx = [x[:zone1_K] for x in zone2_idx]

zone1_idx = np.array(zone1_idx)
zone2_idx = np.array(zone2_idx)

In [ ]:
for idx in zone2_idx:
    pca = PCA(n_components=8)
    subX = X[idx].copy()
    subX[:, ~spec_good_mask] = 0.
    
    subX_pca = pca.fit_transform(subX)
    subX_patched = pca.inverse_transform(subX_pca)
    
    break

In [ ]:
plt.plot(np.std(subX, axis=0))